# Dynamic Economic Emissions Dispatch (DEED) via Reinforcement Learning

This notebook demonstrates how to solve the **Dynamic Economic Emissions Dispatch** (DEED) problem using modern reinforcement learning (PPO from Stable Baselines3).

## Problem Definition

The DEED problem is a multi-objective optimization problem in power systems engineering. Given a set of $N$ thermal generating units and a 24-hour demand profile, the goal is to determine the power output of each generator at each hour to **minimize both fuel cost and pollutant emissions** while satisfying:

1. **Power balance constraint**: Total generation must meet demand plus transmission losses
2. **Generation limits**: $P_i^{\min} \leq P_i^m \leq P_i^{\max}$
3. **Ramp rate limits**: $-DR_i \leq P_i^m - P_i^{m-1} \leq UR_i$

### Cost Function

$$F_C = \sum_{m=1}^{M} \sum_{i=1}^{N} \left[ a_i + b_i P_i^m + c_i (P_i^m)^2 + |d_i \sin(e_i (P_i^{\min} - P_i^m))| \right]$$

The sinusoidal term models the **valve-point loading effect** in steam turbines.

### Emissions Function

$$F_E = \sum_{m=1}^{M} \sum_{i=1}^{N} E \left[ \alpha_i + \beta_i P_i^m + \gamma_i (P_i^m)^2 + \eta_i \exp(\delta_i P_i^m) \right]$$

### Combined Objective (CEED)

Using linear scalarization with weights $W_C$ and $W_E$:

$$\min \; W_C \cdot F_C + W_E \cdot F_E$$

subject to power balance, generation limits, and ramp rate constraints.

## Test System

We use a standard **IEEE 10-generator benchmark** system with:
- 10 thermal generating units with known cost/emission coefficients
- Kron's B-matrix for transmission loss calculation
- 24-hour load demand profile (peak: 2150 MW at hour 12)


---
## Section 2: Environment Setup

We use a custom Gymnasium environment (`deed_env.py`) that models the DEED problem. The agent controls 9 generators (units 2-10) sequentially; unit 1 is the "slack" generator determined by the power balance equation.


In [ ]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

# Import the DEED environment
from deed_env import DEEDEnv

print("Environment module loaded successfully.")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")


### Create and Explore the Environment

In [ ]:
# Create the environment
env = DEEDEnv(Wc=0.5, We=0.5)
obs, info = env.reset(seed=42)

print("=== DEED Environment ===")
print(f"Observation space: {env.observation_space}")
print(f"Action space: {env.action_space}")
print(f"Number of generators: {env.N}")
print(f"Scheduling horizon: {env.M} hours")
print(f"Steps per episode: {env.M * (env.N - 1)} (24 hours x 9 generators)")
print()
print(f"Initial observation (14-dim vector):")
print(f"  {obs}")
print()
print(f"Initial info: {info}")


### Generator Characteristics

In [ ]:
# Display generator data
gen_cols = ["P_min", "P_max", "a", "b", "c", "d", "e",
            "alpha", "beta", "gamma", "eta", "delta", "UR", "DR"]
gen_df = pd.DataFrame(env.GEN_DATA, columns=gen_cols,
                      index=[f"Unit {i+1}" for i in range(10)])
print("Generator Characteristics (IEEE 10-Unit Benchmark):")
display(gen_df[["P_min", "P_max", "UR", "DR"]].astype(int))


### 24-Hour Demand Profile

In [ ]:
# Plot the demand profile
fig, ax = plt.subplots(figsize=(10, 4))
hours = np.arange(1, 25)
ax.bar(hours, env.DEMAND, color='steelblue', alpha=0.8, edgecolor='navy')
ax.set_xlabel('Hour', fontsize=12)
ax.set_ylabel('Power Demand (MW)', fontsize=12)
ax.set_title('24-Hour Power Demand Profile', fontsize=14)
ax.set_xticks(hours)
ax.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.savefig('demand_profile.png', dpi=150, bbox_inches='tight')
plt.show()
print(f"Peak demand: {env.DEMAND.max():.0f} MW at hour {np.argmax(env.DEMAND)+1}")
print(f"Minimum demand: {env.DEMAND.min():.0f} MW at hour {np.argmin(env.DEMAND)+1}")


---
## Section 3: Training with PPO (Proximal Policy Optimization)

We use **Stable Baselines3's PPO** algorithm to train an RL agent for the DEED problem. PPO is a modern policy gradient method that is well-suited for environments with continuous or large discrete action spaces.

This replaces the original tabular Q-learning approach, which was fundamentally inadequate for this problem due to the enormous state-action space (216 steps x 101 actions).


In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.callbacks import BaseCallback

class RewardLoggerCallback(BaseCallback):
    """Custom callback to log episode rewards during training."""
    def __init__(self, verbose=0):
        super().__init__(verbose)
        self.episode_rewards = []
        self.episode_costs = []
        self.episode_emissions = []
        self._current_reward = 0.0

    def _on_step(self):
        # Check if episode ended
        infos = self.locals.get("infos", [{}])
        dones = self.locals.get("dones", [False])
        for i, done in enumerate(dones):
            if done and "total_cost" in infos[i]:
                self.episode_rewards.append(infos[i].get("total_cost", 0) + infos[i].get("total_emissions", 0))
                self.episode_costs.append(infos[i].get("total_cost", 0))
                self.episode_emissions.append(infos[i].get("total_emissions", 0))
        return True

print("Stable Baselines3 imported successfully.")


### Train the PPO Agent

In [ ]:
# Training configuration
TOTAL_TIMESTEPS = 50_000  # Increase for better results (e.g., 200_000)
SEED = 42

# Create environment
train_env = DEEDEnv(Wc=0.5, We=0.5)

# Create PPO agent
model = PPO(
    "MlpPolicy",
    train_env,
    learning_rate=3e-4,
    n_steps=216 * 4,      # Collect 4 full episodes before update
    batch_size=216,        # One full episode per batch
    n_epochs=10,
    gamma=0.99,
    gae_lambda=0.95,
    clip_range=0.2,
    ent_coef=0.01,
    verbose=1,
    seed=SEED,
)

# Train with callback
callback = RewardLoggerCallback()
print(f"Training PPO for {TOTAL_TIMESTEPS:,} timesteps...")
print(f"This is approximately {TOTAL_TIMESTEPS // 216} episodes.")
print()
model.learn(total_timesteps=TOTAL_TIMESTEPS, callback=callback, progress_bar=False)
print()
print(f"Training complete. Episodes logged: {len(callback.episode_costs)}")


### Training Curves

In [ ]:
# Plot training progress
if len(callback.episode_costs) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    # Smoothing function
    def smooth(y, window=10):
        if len(y) < window:
            return y
        return pd.Series(y).rolling(window=window, min_periods=1).mean().values

    episodes = np.arange(1, len(callback.episode_costs) + 1)

    # Cost curve
    axes[0].plot(episodes, callback.episode_costs, alpha=0.3, color='blue', label='Raw')
    axes[0].plot(episodes, smooth(callback.episode_costs), color='blue', linewidth=2, label='Smoothed')
    axes[0].set_xlabel('Episode', fontsize=12)
    axes[0].set_ylabel('Total Fuel Cost ($/day)', fontsize=12)
    axes[0].set_title('Training: Total Fuel Cost', fontsize=14)
    axes[0].legend()
    axes[0].grid(alpha=0.3)

    # Emissions curve
    axes[1].plot(episodes, callback.episode_emissions, alpha=0.3, color='green', label='Raw')
    axes[1].plot(episodes, smooth(callback.episode_emissions), color='green', linewidth=2, label='Smoothed')
    axes[1].set_xlabel('Episode', fontsize=12)
    axes[1].set_ylabel('Total Emissions (tons/day)', fontsize=12)
    axes[1].set_title('Training: Total Emissions', fontsize=14)
    axes[1].legend()
    axes[1].grid(alpha=0.3)

    plt.tight_layout()
    plt.savefig('training_curves.png', dpi=150, bbox_inches='tight')
    plt.show()
else:
    print("No episode data logged during training.")


---
## Section 4: Results Analysis

Evaluate the trained PPO agent and visualize the dispatch schedule.


In [ ]:
# Evaluate the trained agent
eval_env = DEEDEnv(Wc=0.5, We=0.5)
obs, info = eval_env.reset(seed=99)

total_reward = 0.0
terminated = False
while not terminated:
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = eval_env.step(action)
    total_reward += reward

print("=== PPO Agent Evaluation ===")
print(f"Total reward: {total_reward:,.2f}")
print(f"Total fuel cost: {info['total_cost']:,.2f} $/day")
print(f"Total emissions: {info['total_emissions']:,.2f} tons/day")
print()

# Get dispatch schedule
schedule_df = eval_env.get_dispatch_schedule()
print("Dispatch Schedule (MW):")
display(schedule_df.round(1))


### Dispatch Schedule Visualization

In [ ]:
# Plot dispatch schedule as stacked area chart
fig, ax = plt.subplots(figsize=(14, 6))
hours = np.arange(1, 25)
schedule = eval_env.p_schedule  # (10, 24)

# Stacked area chart
bottom = np.zeros(24)
colors = plt.cm.Set3(np.linspace(0, 1, 10))
for i in range(10):
    power = np.maximum(schedule[i], 0)  # Avoid negative in plot
    ax.bar(hours, power, bottom=bottom, color=colors[i], 
           label=f'Unit {i+1}', alpha=0.85, width=0.8)
    bottom += power

ax.plot(hours, eval_env.DEMAND, 'k--', linewidth=2, label='Demand', marker='o', markersize=4)
ax.set_xlabel('Hour', fontsize=12)
ax.set_ylabel('Power (MW)', fontsize=12)
ax.set_title('Optimal Dispatch Schedule (PPO Agent)', fontsize=14)
ax.legend(bbox_to_anchor=(1.02, 1), loc='upper left', fontsize=9)
ax.set_xticks(hours)
ax.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.savefig('dispatch_schedule.png', dpi=150, bbox_inches='tight')
plt.show()


### Hourly Cost and Emissions

In [ ]:
# Plot hourly cost and emissions
hourly_costs = eval_env.get_hourly_costs()
hourly_emissions = eval_env.get_hourly_emissions()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
hours = np.arange(1, 25)

axes[0].bar(hours, hourly_costs, color='coral', alpha=0.8, edgecolor='darkred')
axes[0].set_xlabel('Hour', fontsize=12)
axes[0].set_ylabel('Fuel Cost ($)', fontsize=12)
axes[0].set_title('Hourly Fuel Cost', fontsize=14)
axes[0].set_xticks(hours)
axes[0].grid(axis='y', alpha=0.3)

axes[1].bar(hours, hourly_emissions, color='seagreen', alpha=0.8, edgecolor='darkgreen')
axes[1].set_xlabel('Hour', fontsize=12)
axes[1].set_ylabel('Emissions (tons)', fontsize=12)
axes[1].set_title('Hourly Emissions', fontsize=14)
axes[1].set_xticks(hours)
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('hourly_cost_emissions.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"Total daily cost: {hourly_costs.sum():,.2f} $")
print(f"Total daily emissions: {hourly_emissions.sum():,.2f} tons")


### Comparison with Equal Dispatch Baseline

In [ ]:
# Equal dispatch baseline: each generator outputs equal share of demand
baseline_env = DEEDEnv(Wc=0.5, We=0.5)
baseline_env.reset(seed=99)

for m in range(24):
    demand = baseline_env.DEMAND[m]
    total_capacity = sum(baseline_env.GEN_DATA[n, baseline_env.P_MAX] for n in range(10))
    for n in range(10):
        p_min = baseline_env.GEN_DATA[n, baseline_env.P_MIN]
        p_max = baseline_env.GEN_DATA[n, baseline_env.P_MAX]
        # Proportional dispatch based on capacity
        share = (p_max / total_capacity) * demand
        power = np.clip(share, p_min, p_max)
        baseline_env.p_schedule[n, m] = power

baseline_cost = sum(baseline_env._fuel_cost_total(m) for m in range(24))
baseline_emissions = sum(baseline_env._emissions_total(m) for m in range(24))

print("=== Comparison ===")
print(f"{'Method':<25} {'Cost ($)':>15} {'Emissions (tons)':>20}")
print("-" * 62)
print(f"{'PPO Agent':<25} {info['total_cost']:>15,.2f} {info['total_emissions']:>20,.2f}")
print(f"{'Proportional Dispatch':<25} {baseline_cost:>15,.2f} {baseline_emissions:>20,.2f}")
if info['total_cost'] < baseline_cost:
    pct = (1 - info['total_cost']/baseline_cost) * 100
    print(f"\nPPO achieves {pct:.1f}% cost reduction over proportional dispatch.")


---
## Section 5: Multi-Objective Pareto Front Analysis

This section presents **new research**: by training multiple PPO agents with different weight combinations $(W_C, W_E)$, we can trace out the **Pareto front** of the Combined Economic and Emissions Dispatch (CEED) problem.

The Pareto front shows the trade-off between fuel cost and emissions -- solutions where improving one objective necessarily worsens the other. This is a key contribution: using RL to approximate the Pareto front without requiring traditional mathematical programming.

### Methodology
1. Choose a set of weight pairs: $(W_C, W_E) \in \{(1.0, 0.0), (0.8, 0.2), ..., (0.0, 1.0)\}$
2. Train a PPO agent for each weight pair
3. Evaluate each agent and record total cost and emissions
4. Plot the Pareto front


In [ ]:
# Pareto front analysis
# Define weight combinations (Wc + We = 1.0)
weight_pairs = [
    (1.0, 0.0),   # Pure cost minimization
    (0.9, 0.1),
    (0.8, 0.2),
    (0.7, 0.3),
    (0.6, 0.4),
    (0.5, 0.5),   # Equal weighting
    (0.4, 0.6),
    (0.3, 0.7),
    (0.2, 0.8),
    (0.1, 0.9),
    (0.0, 1.0),   # Pure emissions minimization
]

PARETO_TIMESTEPS = 30_000  # Timesteps per weight pair (increase for better results)

pareto_results = []

print(f"Training {len(weight_pairs)} agents for Pareto front analysis...")
print(f"Timesteps per agent: {PARETO_TIMESTEPS:,}")
print()

for idx, (wc, we) in enumerate(weight_pairs):
    print(f"[{idx+1}/{len(weight_pairs)}] Wc={wc:.1f}, We={we:.1f} ... ", end="", flush=True)
    
    # Create environment with these weights
    pareto_env = DEEDEnv(Wc=wc, We=we)
    
    # Train PPO agent
    pareto_model = PPO(
        "MlpPolicy",
        pareto_env,
        learning_rate=3e-4,
        n_steps=216 * 4,
        batch_size=216,
        n_epochs=10,
        gamma=0.99,
        ent_coef=0.01,
        verbose=0,
        seed=42,
    )
    pareto_model.learn(total_timesteps=PARETO_TIMESTEPS, progress_bar=False)
    
    # Evaluate (best of 3 runs)
    best_cost = float('inf')
    best_emissions = float('inf')
    best_combined = float('inf')
    
    for eval_seed in [99, 100, 101]:
        eval_env = DEEDEnv(Wc=wc, We=we)
        obs, _ = eval_env.reset(seed=eval_seed)
        terminated = False
        while not terminated:
            action, _ = pareto_model.predict(obs, deterministic=True)
            obs, _, terminated, _, eval_info = eval_env.step(action)
        
        cost = eval_info['total_cost']
        emissions = eval_info['total_emissions']
        combined = wc * cost + we * emissions
        
        if combined < best_combined:
            best_cost = cost
            best_emissions = emissions
            best_combined = combined
    
    pareto_results.append({
        'Wc': wc,
        'We': we,
        'cost': best_cost,
        'emissions': best_emissions,
    })
    print(f"Cost={best_cost:,.0f}, Emissions={best_emissions:,.0f}")

print("\nPareto front analysis complete!")


### Pareto Front Visualization

In [ ]:
# Plot the Pareto front
costs = [r['cost'] for r in pareto_results]
emissions = [r['emissions'] for r in pareto_results]
wcs = [r['Wc'] for r in pareto_results]

fig, ax = plt.subplots(figsize=(10, 7))

# Plot Pareto front
scatter = ax.scatter(costs, emissions, c=wcs, cmap='RdYlGn', s=120,
                     edgecolors='black', linewidth=1.5, zorder=5)
ax.plot(costs, emissions, 'k--', alpha=0.5, linewidth=1, zorder=3)

# Annotate points with weight values
for i, r in enumerate(pareto_results):
    offset_x = (max(costs) - min(costs)) * 0.02
    offset_y = (max(emissions) - min(emissions)) * 0.02
    ax.annotate(f"({r['Wc']:.1f}, {r['We']:.1f})",
                (r['cost'], r['emissions']),
                textcoords="offset points", xytext=(8, 8),
                fontsize=8, alpha=0.8)

cbar = plt.colorbar(scatter, ax=ax, label='Cost Weight (Wc)')
ax.set_xlabel('Total Fuel Cost ($/day)', fontsize=13)
ax.set_ylabel('Total Emissions (tons/day)', fontsize=13)
ax.set_title('Pareto Front: Cost vs Emissions Trade-off (DEED via RL)', fontsize=14)
ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('pareto_front.png', dpi=150, bbox_inches='tight')
plt.show()

# Print results table
print("\n=== Pareto Front Results ===")
print(f"{'Wc':>5} {'We':>5} {'Cost ($)':>15} {'Emissions (tons)':>20}")
print("-" * 50)
for r in pareto_results:
    print(f"{r['Wc']:>5.1f} {r['We']:>5.1f} {r['cost']:>15,.2f} {r['emissions']:>20,.2f}")


### Analysis of the Pareto Front

The Pareto front reveals several key insights:

1. **Trade-off structure**: There is a clear inverse relationship between cost and emissions. Cheaper dispatch schedules produce more emissions, and vice versa.

2. **Diminishing returns**: The marginal cost of emissions reduction increases as we move toward the emissions-minimized end of the front.

3. **Decision support**: The Pareto front provides decision-makers with a menu of optimal operating points. The "knee" of the front (around $W_C = 0.5, W_E = 0.5$) often represents a good compromise.

4. **RL advantage**: Unlike traditional methods (weighted sum LP/QP, epsilon-constraint), the RL approach naturally handles the non-convex valve-point loading effects and can be extended to more complex constraints.

### Future Work

- Increase training timesteps for more converged Pareto points
- Use multi-objective RL algorithms (e.g., MORL) for direct Pareto optimization
- Add renewable energy sources and battery storage to the environment
- Compare RL results with classical optimization benchmarks (PSO, DE, NSGA-II)


In [ ]:
# Summary statistics
print("=== DEED Notebook Summary ===")
print(f"Environment: {env.N} generators, {env.M} hours")
print(f"Steps per episode: {env.M * (env.N - 1)}")
print(f"Pareto front points: {len(pareto_results)}")
print()
print("Files created:")
print("  - deed_env.py: Gymnasium DEED environment")
print("  - tests/test_deed_env.py: 37 unit tests")
print("  - requirements.txt: Python dependencies")
print("  - DEED.ipynb: This notebook")
print()
print("To run tests: pytest tests/ -v")
print("To run notebook: jupyter notebook DEED.ipynb")
